In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from FinMind.data import DataLoader
import helper
api = DataLoader()

d:\Practice\stock_trade\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [17]:
today = datetime.today() #datetime.strptime('2025-04-30', "%Y-%m-%d")
start_date = (today - relativedelta(months=24)).strftime("%Y-%m-%d")
end_date = today.strftime("%Y-%m-%d")
print('start date', start_date)
print('end date', end_date)

ticker = '4541'
df = api.taiwan_stock_daily(
    stock_id=ticker,
    start_date=start_date,
    end_date=end_date
)
df.set_index('date', inplace=True)
df.index = pd.to_datetime(df.index)
df = df.rename(columns={'max': 'High', 'min': 'Low', 'open': 'Open', 'close': 'Close', 'Trading_Volume': 'Volume'})
df

2025-10-14 11:41:50.700 | INFO     | FinMind.data.finmind_api:get_data:158 - download Dataset.TaiwanStockPrice, data_id: 4541


start date 2023-10-14
end date 2025-10-14


,stock_id,Volume,Trading_money,Open,High,Low,Close,spread,Trading_turnover
date,,,,,,,,,
2023-10-16,4541,289000,9533500,33.60,33.85,32.60,32.60,-1.00,182
2023-10-17,4541,291000,9407450,33.00,33.00,31.55,31.75,-0.85,196
2023-10-18,4541,571000,17599200,31.75,31.75,30.55,30.70,-1.05,275
2023-10-19,4541,437000,13648500,30.25,31.60,30.25,31.55,0.85,275
2023-10-20,4541,211000,6515050,31.30,31.30,30.70,30.85,-0.70,98
...,...,...,...,...,...,...,...,...,...
2025-10-03,4541,3209941,162769648,49.60,51.60,49.60,50.50,0.95,2419
2025-10-07,4541,1692528,84355871,50.50,50.50,49.30,49.80,-0.70,1676
2025-10-08,4541,1339157,65838388,49.95,50.10,48.65,49.10,-0.70,1199


In [27]:
helper.EMA_KD.n1 = 10
helper.EMA_KD.n2 = 50
bt = Backtest(df, helper.EMA_KD, cash=100_000, commission=.001425)
stats = bt.run()
stats

Backtest.run:   0%|          | 0/476 [00:00<?, ?bar/s]d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=59: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=111: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=440: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


Start                     2023-10-16 00:00:00
End                       2025-10-13 00:00:00
Duration                    728 days 00:00:00
Exposure Time [%]                    35.46392
Equity Final [$]                 165093.44411
Equity Peak [$]                  165093.44411
Commissions [$]                    1349.20589
Return [%]                           65.09344
Buy & Hold Return [%]                56.41026
Return (Ann.) [%]                    29.75668
Volatility (Ann.) [%]                31.03643
CAGR [%]                             18.95098
Sharpe Ratio                          0.95877
Sortino Ratio                         2.45711
Calmar Ratio                           1.5099
Alpha [%]                            50.91834
Beta                                  0.25129
Max. Drawdown [%]                   -19.70768
Avg. Drawdown [%]                    -6.65641
Max. Drawdown Duration      317 days 00:00:00
Avg. Drawdown Duration       79 days 00:00:00
# Trades                          

In [28]:
trades = stats['_trades']

for _, trade in trades.iterrows():
    print(f"Buy date: {trade['EntryTime']}, Sell date: {trade['ExitTime']}")
    print(f"Buy price: {trade['EntryPrice']}, Sell price: {trade['ExitPrice']}, P/L: {trade['ReturnPct']:.2%}")

Buy date: 2023-11-27 00:00:00, Sell date: 2024-04-11 00:00:00
Buy price: 32.75, Sell price: 33.3, P/L: 1.39%
Buy date: 2024-09-25 00:00:00, Sell date: 2024-10-23 00:00:00
Buy price: 29.55, Sell price: 34.4, P/L: 16.10%
Buy date: 2025-01-20 00:00:00, Sell date: 2025-02-07 00:00:00
Buy price: 34.4, Sell price: 35.7, P/L: 3.49%
Buy date: 2025-06-17 00:00:00, Sell date: 2025-09-04 00:00:00
Buy price: 48.4, Sell price: 65.8, P/L: 35.61%


In [24]:
helper.EMA_VWAP_KD.n1 = 10
helper.EMA_VWAP_KD.n2 = 50
bt = Backtest(df, helper.EMA_VWAP_KD, cash=100_000, commission=.001425)
stats = bt.run()
stats

Backtest.run:   0%|          | 0/476 [00:00<?, ?bar/s]d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=59: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=111: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


Start                     2023-10-16 00:00:00
End                       2025-10-13 00:00:00
Duration                    728 days 00:00:00
Exposure Time [%]                    31.75258
Equity Final [$]                 142223.33214
Equity Peak [$]                  142223.33214
Commissions [$]                     933.11786
Return [%]                           42.22333
Buy & Hold Return [%]                56.41026
Return (Ann.) [%]                    20.08306
Volatility (Ann.) [%]                26.89192
CAGR [%]                             12.96696
Sharpe Ratio                          0.74681
Sortino Ratio                         1.71674
Calmar Ratio                          1.01906
Alpha [%]                            29.68286
Beta                                  0.22231
Max. Drawdown [%]                   -19.70747
Avg. Drawdown [%]                     -7.5244
Max. Drawdown Duration      414 days 00:00:00
Avg. Drawdown Duration      108 days 00:00:00
# Trades                          

In [7]:
trades = stats['_trades']

for _, trade in trades.iterrows():
    print(f"Buy date: {trade['EntryTime']}, Sell date: {trade['ExitTime']}")
    print(f"Buy price: {trade['EntryPrice']}, Sell price: {trade['ExitPrice']}, P/L: {trade['ReturnPct']:.2%}")

Buy date: 2023-12-11 00:00:00, Sell date: 2024-01-11 00:00:00
Buy price: 183.0, Sell price: 189.5, P/L: 3.26%
Buy date: 2024-05-06 00:00:00, Sell date: 2024-05-24 00:00:00
Buy price: 232.5, Sell price: 291.0, P/L: 24.84%


In [6]:
helper.SmaCross.n1 = 5
helper.SmaCross.n2 = 20
bt = Backtest(df, helper.SmaCross, cash=100_000, commission=.001425)
stats = bt.run()
stats

Start                     2023-10-16 00:00:00
End                       2025-10-13 00:00:00
Duration                    728 days 00:00:00
Exposure Time [%]                    44.12371
Equity Final [$]                 102577.17254
Equity Peak [$]                  111044.55766
Commissions [$]                    3678.42745
Return [%]                            2.57717
Buy & Hold Return [%]                19.87642
Return (Ann.) [%]                     1.33088
Volatility (Ann.) [%]                31.38011
CAGR [%]                              0.88469
Sharpe Ratio                          0.04241
Sortino Ratio                         0.07376
Calmar Ratio                          0.03431
Alpha [%]                           -10.94965
Beta                                  0.68055
Max. Drawdown [%]                   -38.78839
Avg. Drawdown [%]                    -7.42658
Max. Drawdown Duration      543 days 00:00:00
Avg. Drawdown Duration       77 days 00:00:00
# Trades                          

In [25]:
helper.SMA_KD.n1 = 10
helper.SMA_KD.n2 = 50
bt = Backtest(df, helper.SMA_KD, cash=100_000, commission=.001425)
stats = bt.run()
stats

Backtest.run:   0%|          | 0/435 [00:00<?, ?bar/s]d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=64: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=112: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(


Start                     2023-10-16 00:00:00
End                       2025-10-13 00:00:00
Duration                    728 days 00:00:00
Exposure Time [%]                    16.49485
Equity Final [$]                   118524.102
Equity Peak [$]                  120888.83018
Commissions [$]                      1274.148
Return [%]                            18.5241
Buy & Hold Return [%]                58.18476
Return (Ann.) [%]                     9.23179
Volatility (Ann.) [%]                14.29275
CAGR [%]                              6.05923
Sharpe Ratio                          0.64591
Sortino Ratio                         1.34389
Calmar Ratio                          0.75955
Alpha [%]                            14.09873
Beta                                  0.07606
Max. Drawdown [%]                   -12.15427
Avg. Drawdown [%]                    -6.72199
Max. Drawdown Duration      250 days 00:00:00
Avg. Drawdown Duration      122 days 00:00:00
# Trades                          

In [8]:
trades = stats['_trades']

for _, trade in trades.iterrows():
    print(f"Buy date: {trade['EntryTime']}, Sell date: {trade['ExitTime']}")
    print(f"Buy price: {trade['EntryPrice']}, Sell price: {trade['ExitPrice']}, P/L: {trade['ReturnPct']:.2%}")

Buy date: 2024-03-05 00:00:00, Sell date: 2024-04-03 00:00:00
Buy price: 49.9, Sell price: 54.6, P/L: 9.12%
Buy date: 2023-11-21 00:00:00, Sell date: 2024-04-03 00:00:00
Buy price: 51.5, Sell price: 54.6, P/L: 5.73%
Buy date: 2024-12-25 00:00:00, Sell date: 2025-09-02 00:00:00
Buy price: 44.25, Sell price: 61.0, P/L: 37.51%
Buy date: 2024-05-21 00:00:00, Sell date: 2025-09-02 00:00:00
Buy price: 57.6, Sell price: 61.0, P/L: 5.61%


In [26]:
helper.BOLL_KD30.buy_strategy = helper.BuyStrategy.BOLL_KD30
bt = Backtest(df, helper.BOLL_KD30, cash=100_000, commission=.001425)
stats = bt.run()
stats

Backtest.run:   0%|          | 0/465 [00:00<?, ?bar/s]d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=68: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=69: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=100: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=101: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\Practice\stock_trade\.venv\Lib\site-packages\backtesting\backtesting.py:966: UserWarning: time=190: Broker canceled the relative-sized order due to insufficient margin.
  warnings.warn(
d:\

Start                     2023-10-16 00:00:00
End                       2025-10-13 00:00:00
Duration                    728 days 00:00:00
Exposure Time [%]                    42.47423
Equity Final [$]                 133005.22637
Equity Peak [$]                  137260.82212
Commissions [$]                    1295.47363
Return [%]                           33.00523
Buy & Hold Return [%]                62.12625
Return (Ann.) [%]                      15.974
Volatility (Ann.) [%]                27.27859
CAGR [%]                             10.37676
Sharpe Ratio                          0.58559
Sortino Ratio                         1.11748
Calmar Ratio                          0.75428
Alpha [%]                            17.21456
Beta                                  0.25417
Max. Drawdown [%]                   -21.17774
Avg. Drawdown [%]                    -4.90183
Max. Drawdown Duration      221 days 00:00:00
Avg. Drawdown Duration       34 days 00:00:00
# Trades                          

In [23]:
trades = stats['_trades']

for _, trade in trades.iterrows():
    print(f"Buy date: {trade['EntryTime']}, Sell date: {trade['ExitTime']}")
    print(f"Buy price: {trade['EntryPrice']}, Sell price: {trade['ExitPrice']}, P/L: {trade['ReturnPct']:.2%}")

Buy date: 2024-04-23 00:00:00, Sell date: 2024-05-29 00:00:00
Buy price: 111.0, Sell price: 119.5, P/L: 7.36%
Buy date: 2024-06-12 00:00:00, Sell date: 2024-07-17 00:00:00
Buy price: 107.5, Sell price: 113.5, P/L: 5.29%
Buy date: 2024-07-23 00:00:00, Sell date: 2024-10-14 00:00:00
Buy price: 102.0, Sell price: 109.5, P/L: 7.06%
Buy date: 2024-12-17 00:00:00, Sell date: 2025-01-21 00:00:00
Buy price: 106.5, Sell price: 111.5, P/L: 4.40%
Buy date: 2025-03-31 00:00:00, Sell date: 2025-05-15 00:00:00
Buy price: 96.0, Sell price: 114.0, P/L: 18.44%


In [4]:
bt.plot()

GridPlot(id='p1415', ...)